In [21]:
import os
import numpy as np
import torch
import torchvision
from torchvision import models,transforms, datasets
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from torch.optim import lr_scheduler

# Step 1: Read class.txt file
with open(r"C:\Users\RahulSingh\Downloads\Dataset\Dataset\Dataset\classes.txt", 'r') as file:
    classes = file.read().splitlines()
# Step 2: Prepare your dataset
dataset_path = r"C:\Users\RahulSingh\Downloads\Dataset\Dataset\Dataset\Classification"

# Step 3: Define transformations and create dataset
transform = transforms.Compose([
     transforms.RandomResizedCrop(224),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [22]:
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Step 4: Split your dataset
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Step 5: Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [24]:
# Step 6: Define a model
class DocumentClassifier(nn.Module):
    def __init__(self, num_classes):
        super(DocumentClassifier, self).__init__()
        self.resnet = models.resnet18(weights='IMAGENET1K_V1')  # Load pre-trained ResNet-18 model
        # Replace the last fully connected layer with a new one for our classification task
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)
    def forward(self, x):
        return self.resnet(x)
# Step 7: Initialize the model
model = DocumentClassifier(num_classes=len(classes))

# Step 8: Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [25]:
# Step 9: Train the model
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        exp_lr_scheduler.step()
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")

100%|██████████████████████████████████████████████████████████████████████████████████| 88/88 [14:43<00:00, 10.04s/it]


Epoch [1/5], Loss: 1.5867


100%|██████████████████████████████████████████████████████████████████████████████████| 88/88 [15:27<00:00, 10.54s/it]


Epoch [2/5], Loss: 1.5213


100%|██████████████████████████████████████████████████████████████████████████████████| 88/88 [29:50<00:00, 20.35s/it]


Epoch [3/5], Loss: 1.5251


100%|██████████████████████████████████████████████████████████████████████████████████| 88/88 [29:05<00:00, 19.84s/it]


Epoch [4/5], Loss: 1.5244


100%|██████████████████████████████████████████████████████████████████████████████████| 88/88 [14:18<00:00,  9.76s/it]

Epoch [5/5], Loss: 1.5055


In [26]:
# Step 10: Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [02:19<00:00,  6.33s/it]

Test Accuracy: 0.5308


In [27]:
torch.save(model, 'Resnet_5_epoch.pth')

In [28]:
from PIL import Image
image_path = r"C:\Users\RahulSingh\Downloads\e.jpg" # Replace 'path_to_your_new_image.jpg' with the actual path
image = Image.open(image_path).convert("RGB")
input_image = transform(image).unsqueeze(0)

In [29]:
outputs = model(input_image)
_, predicted = torch.max(outputs, 1)

In [30]:
outputs

tensor([[-3.1381e+00,  2.0085e-03, -1.9436e-01,  2.4960e+00,  5.8384e-01,
         -2.6729e+00, -1.0771e+00,  1.1330e+00, -7.4773e-01, -4.2856e-01]],
       grad_fn=<AddmmBackward0>)

In [31]:
predicted

tensor([3])

In [29]:
classes

['ADVE',
 'Email',
 'Letter',
 'News',
 'Report',
 'Scientific',
 'Form',
 'Memo',
 'Note',
 'Resume']